425 Project

In [7]:
import csv
import sys
import torch
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import pyranges as pr
import sqlite3

import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os 
import warnings
#warnings.filterwarnings("ignore")

In [17]:
def process_dataframe(filename):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filename, sep='\t', header=None)

    # Set column names
    df.columns = ['chromosome', 'start', 'end', 'name', 'score', 'strand', 'stuff', 'things', 'who', 'ahh', 'scream', 'ska', 'die', 'grr']

    # Calculate new positions based on existing start and end positions
    df['dif'] = df['end'] - df['start']
    df['mid'] = df['dif'] // 2
    df['midpoint'] = df['start'] + df['mid']
    df['new_start'] = df['midpoint'] - 300
    df['new_end'] = df['midpoint'] + 300
    df['dif_new'] = df['new_end'] - df['new_start']

    # Create a new DataFrame with selected columns
    df_new = pd.DataFrame({
        'chromosome': df['chromosome'],
        'start': df['start'],
        'end': df['end'],
        'midpoint': df['midpoint'],
        'dif': df['dif'],
        'new_start': df['new_start'], #new start and end are from the midpoint out 250 each way
        'new_end': df['new_end'],
        'position_difference_new': df['dif_new']
    })

    return df

In [29]:
testFile = "Final_bed_files/SRP034156_BedFiles/SRX391990.target.all.bed"
directory = "Final_bed_files//test_bedFiles"

# List to store dataframes
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith(".bed"):
        file_path = os.path.join(directory, filename)
        df = process_dataframe(file_path)
        dataframes.append(df)


In [30]:
#Check size of each dataframe within the list
for i, df in enumerate(dataframes):
    num_rows, num_columns = df.shape
    print(f"DataFrame {i+1}: {num_rows} rows x {num_columns} columns")

DataFrame 1: 19464 rows x 20 columns
DataFrame 2: 22100 rows x 20 columns
DataFrame 3: 16752 rows x 20 columns
DataFrame 4: 22932 rows x 20 columns
